In [53]:
import openai
import os
import re

In [54]:
os.chdir('C:/Users/kaosh/Desktop/Side-project-ML/sentence_analyze')  # Change to your own path

In [55]:
import openpyxl
wb = openpyxl.load_workbook('TCB Response_criteria.xlsx', data_only=True)     # Open Excel file

Domain = 'Innovation'

In [56]:
s1 = wb[Domain+"1"]        # Get the worksheet content

In [57]:
# Store all Capabilities in an array
s1_capability_read = s1.iter_rows(max_col=1)
capability_read = []
for row in s1_capability_read:
    tmp = []
    for column in row:
        tmp.append(column.value)
    capability_read.append(tmp)

# Store all questions in an array
s1_question_read = s1.iter_rows(min_col=2, max_col=2)
question_read = []
for row in s1_question_read:
    tmp = []
    for column in row:
        tmp.append(column.value)
    question_read.append(tmp)

# Store all answer 
s1_answer_read = s1.iter_rows(min_col=3, max_col=3)
answer_read = []
for row in s1_answer_read:
    tmp = []
    for column in row:
        tmp.append(column.value)
    answer_read.append(tmp)
    
# Store all criteria in an array
s1_criteria_read = s1.iter_rows(min_col=4, max_col=4)
criteria_read = []
for row in s1_criteria_read:
    tmp = []
    for column in row:
        tmp.append(column.value)
    criteria_read.append(tmp)


In [58]:
# Set your OpenAI API key
openai.api_key = '...'

In [59]:
# The analyze_sentence function sends a request to the OpenAI API with the given sentence and criteria, and returns the decision made by the model.
def analyze_sentence(sentence, criteria):
    response = openai.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=f"Sentence: {sentence}\nCriteria: {criteria}\nDecision:",
        temperature=0.5,
        max_tokens=100,
        stop=["\n"]
    )
    decision = response.choices[0].text.strip()
    return decision

In [60]:
def summarize_paragraph(paragraph):
    # Use the completion endpoint to summarize the paragraph
    response = openai.completions.create(
      model="gpt-3.5-turbo-instruct",
      prompt=f"Write a concise summary of the following: {paragraph}",
      max_tokens=100,
      temperature=0.5,
      top_p=0,
      frequency_penalty=0,
      presence_penalty=0
    )
    
    # Extract and return the summarized text from the response
    return response.choices[0].text.strip()

In [61]:
# Find out all keywords of the criteria
def criteria_keywords(criteria):
    response = openai.completions.create(
      model="gpt-3.5-turbo-instruct",
      prompt=f"Find keyword or key information of the following: {criteria}",
      max_tokens=100,
      temperature=0.5,
      top_p=0,
      frequency_penalty=0,
      presence_penalty=0
    )
    
    return response.choices[0].text.strip()

In [62]:
def Average(lst): 
    return sum(lst) / len(lst)

In [63]:
keywords_array = []
for i in range(0, len(criteria_read), 2):
    keywords = criteria_keywords(criteria_read[i])
    print(keywords)
    keywords_array.append(keywords)
    keywords_array.append(keywords)

Keyword: Active Integration, Discovery and Learning Process, Cloud Technology Integration, New Technologies, Proactive, Leveraging, Robust, Learning Programs, Discovery Programs, Metrics, Successful Integration, Cloud Features.
Keyword: Structured Deployment Process, Cloud Technologies, Effective Use, Impact, Adoption, Processes, Incubate, Structured Programs, Idea Discovery, Full Deployment, Cloud Innovations.
Keyword: Clearly Defined Strategy, Communication, Departmental Understanding, Innovation Strategy, Executive Buy-in, Strategic Alignment, Execution, Innovation Vision, Fully Supported, Unified Approach.
Keyword: Integration, Feedback Loop, Insights, Improve, Innovation Functions, Daily Operations, Meetings, Impact, Closed Feedback Loop, Data Insights, Refine, Improve.
Keyword: Departmental involvement, investment strategies, evaluation process, organizational approach, startup evaluation, multiple departments, structured investment program, criteria, strategic, profitable select

In [64]:
# The main function prompts the user to input the sentence and criteria they want to analyze, calls analyze_sentence with these inputs, and prints out the decision.
run_times = 10 # 期望跑的次數
result = [[] for _ in range(len(answer_read)+1)]
result[0].append("Capability")
result[0].append("Questions")
result[0].append("All Answers concatenate")
result[0].append("Criteria")
result[0].append("Keywords of criteria")
for run_time in range(run_times):
    result[0].append(str(run_time + 1))
result[0].append("Average Score")
result[0].append("Final Justification")
for i in range(len(answer_read)):
    result[i+1].append(capability_read[i][0]) # Put capability array in result
    result[i+1].append(question_read[i][0]) # Put Questions array in result
    result[i+1].append(answer_read[i][0])   # Put all answer array in result
    result[i+1].append(criteria_read[i][0])   # Put all criteria array in result 
    result[i+1].append(keywords_array[i])   # Put all keywords array in result 
    
    All_justification = "" # Store all justification made by GPT
    for run_time in range(run_times):
        if answer_read[i][0] == None:
            result[i+1].append("")
        else: 
            if answer_read[i][0].strip("/") == "":
                result[i+1].append("")
            else:
                sentence = answer_read[i][0]
                criteria = "What percentage is the sentences meet the criteria?" + keywords_array[i]
                decision = analyze_sentence(sentence, criteria)
                result[i+1].append(f'{decision}')
                All_justification = All_justification + f'{decision}' + "/" # Concatenance all justification into a paragraph 
                #print(f'{decision}')
    temp = re.findall(r'\d+%', All_justification)
    res = []
    for n in temp:
        res.append(int(n.strip('%')))
    if len(res) == 0:
        result[i+1].append("")
    else:
        result[i+1].append(Average(res))
        print(Average(res))
    if All_justification == "":
        result[i+1].append("")
    else:
        summary= summarize_paragraph(All_justification)
        result[i+1].append(summary)
        print(summary)

72.5
Approximately 80% of the sentences meet the criteria, discussing active integration, discovery and learning process, cloud technology integration, leveraging new technologies, robust learning programs, and successful integration. However, some sentences do not directly mention all of the keywords, such as proactive approach, metrics, and specific cloud features.
83.75
The sentences cover the majority of the criteria, including a defined strategy, communication through various methods, departmental understanding, executive buy-in and support, strategic alignment, and execution. The only aspect that may be missing is a specific mention of the innovation vision, but it can be inferred from the mention of the cloud vision and strategy. Overall, the sentences effectively convey the importance of clear communication and support for the innovation strategy.
87.0
Approximately 90% of the sentences meet the criteria by mentioning the integration of feedback loops, data insights, and regula

In [65]:
# Save the results in excel '...'
excel = openpyxl.Workbook(Domain+'_Result.xlsx')     
s3 = excel.create_sheet() 
for w in result:
    s3.append(w)
excel.save(Domain+'_Result.xlsx')